## Code for Runoff analysis over CONUS404 based estimation in comparison with USGS-CAMELS datasets

### Author: Miguel Diaz

#### **Libraries**

!pip install absl-py apache-beam xarray xarray-beam earthengine-api geemap pyproj geopandas pandas numpy sparse hvplot holoviews dask cf-xarray geoviews pynhd hydrofunctions intake cartopy shapely matplotlib contextily scipy rasterio requests IPython
!pip install notebook
!earthengine authenticate
!pip install dask[distributed]
!pip install bokeh
!pip install xee
!pip install datashader

!pip install zarr
!pip install fsspec gcsfs s3fs
!pip install --upgrade intake
!pip install zarr intake intake-xarray fsspec s3fs gcsfs
!pip install --upgrade intake intake-xarray zarr
!pip install metpy

!pip install --upgrade xarray metpy
!pip install scikit-learn
!pip install apache-beam
!pip install cf_xarray

In [17]:
import logging
import os
import time
import warnings
import webbrowser

# Data handling and computation
import numpy as np
import pandas as pd
import xarray as xr
import cf_xarray
import dask
import dask.array as da
from dask import delayed
from dask.distributed import Client
import sparse

# Scientific libraries
import pyproj
from scipy.stats import linregress, gumbel_r

# GIS and remote sensing
import geopandas as gpd
import geemap
import ee
import xarray_beam as xbeam
import apache_beam as beam
import xee
import zarr

# Mapping and visualization
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
from holoviews.operation.datashader import rasterize
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import contextily as ctx
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import plotly.express as px

# Geospatial libraries
from shapely.geometry import Polygon
from rasterio.enums import Resampling

# Hydrology and water data
from pynhd import NLDI, WaterData
import hydrofunctions as hf

# Data intake and registry
import intake
print(intake.registry)

# Meteorology and weather data
from metpy.units import units
import metpy  # Ensure this is imported to extend xarray

# Web and API requests
import requests
from IPython.display import HTML

# ABSL (Google's command-line flags and app framework)
from absl import app, flags

# Ensure inline plotting for Jupyter notebooks
%matplotlib inline


<Intake driver registry>


#### **CONUS 404 - SOURCES**

This section contains notebooks that demonstrate how to access and perform basic data manipulation for the [CONUS404 dataset](https://doi.org/10.5066/P9PHPK4F). The examples can also be applied to the [CONUS404 bias-adjusted dataset](https://doi.org/10.5066/P9JE61P7).

In the CONUS404 intake sub-catalog (see [here](../dataset_catalog/README.md) for an explainer of our intake data catalog), you will see entries for four CONUS404 datasets: `conus404-hourly`, `conus404-daily`, `conus404-monthly`, and `conus404-daily-diagnostic` data, as well as two CONUS404 bias-adjusted datasets: `conus404-hourly-ba`, `conus404-daily-ba`. Each of these datasets is duplicated in up to three different storage locations (as the [intake catalog section](../dataset_catalog/README.md) also describes).

In the CONUS404 intake sub-catalog (see [here](../dataset_catalog/README.md) for an explainer of our intake data catalog), you will see entries for:
- four CONUS404 datasets: `conus404-hourly`, `conus404-daily`, `conus404-monthly`, and `conus404-daily-diagnostic` data
- two CONUS404 bias-adjusted datasets: `conus404-hourly-ba`, `conus404-daily-ba`
- two CONUS404 PGW datasets: `conus404-pgw-hourly` and `conus404-pgw-daily-diagnostic`

#### **Dask client with specified number of workers**

In [18]:
client = Client(n_workers=4)  
client

C:\Users\adi10136\AppData\Local\anaconda3\envs\CONUS404_test\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56006 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:56006/status,
Dashboard: http://127.0.0.1:56006/status,Workers: 4
Total threads: 32,Total memory: 127.69 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56009,Workers: 4
Dashboard: http://127.0.0.1:56006/status,Total threads: 32
Started: Just now,Total memory: 127.69 GiB
Comm: tcp://127.0.0.1:56029,Total threads: 8
Dashboard: http://127.0.0.1:56033/status,Memory: 31.92 GiB
Nanny: tcp://127.0.0.1:56012,


In [19]:
import pandas as pd

# File paths
sanford_file = r"C:\Users\adi10136\OneDrive - Iowa State University\Desktop\CONUS404\ET Sanford\ET_Mean_Sanford_Agreed_Basins\AET_Basins_Mean_Values.csv"
conus404_file = r"C:\Users\adi10136\OneDrive - Iowa State University\Desktop\CONUS404\ET_WaterYears\AET_mean_Basins_CONUS404\ET_Basins_Mean_Values.csv"

# Load the CSV files
sanford_df = pd.read_csv(sanford_file)
conus404_df = pd.read_csv(conus404_file)

# Display first few rows
display(sanford_df.head())
display(conus404_df.head())


,name,AET_1990,AET_1991,AET_1992,AET_1993,AET_1994,AET_1995,AET_1996,AET_1997,AET_1998,...,AET_2000,AET_2001,AET_2002,AET_2003,AET_2004,AET_2005,AET_2006,AET_2007,AET_2008,AET_2009
0,1013500,532.240341,507.776588,535.722459,539.077791,565.502197,482.285055,566.518196,523.279233,563.964709,...,537.297215,508.036718,514.742219,564.394825,571.197801,562.928791,590.108365,541.385195,581.603163,555.641039
1,1019000,567.941718,582.810206,511.858339,486.392273,502.745313,517.294172,550.425525,542.522675,534.586589,...,520.603610,478.210596,530.352665,526.827658,563.399872,603.767711,645.874811,527.944763,578.211589,585.123058
2,1022500,569.082703,594.823793,492.098908,469.906591,509.816766,511.827485,546.341557,520.660206,522.562560,...,502.030355,469.466367,546.232928,517.376626,549.466122,563.798092,628.977870,544.564227,569.853500,585.333294
3,1030500,567.746502,532.820693,500.170837,510.349288,524.644646,484.188340,566.161814,533.798302,547.532335,...,522.492560,488.804965,519.639529,550.728153,545.936762,577.533433,618.855820,526.947449,571.365293,557.937971
4,1031500,579.718110,555.205357,499.591072,510.097644,548.800909,465.343476,571.734233,534.114904,578.778529,...,528.061480,502.142351,508.331367,543.141734,560.314031,593.353146,615.236319,522.362770,574.702154,562.149587


,name,ET_1990,ET_1991,ET_1992,ET_1993,ET_1994,ET_1995,ET_1996,ET_1997,ET_1998,...,ET_2000,ET_2001,ET_2002,ET_2003,ET_2004,ET_2005,ET_2006,ET_2007,ET_2008,ET_2009
0,1013500,495.528522,512.796539,463.514122,487.558458,504.996250,516.737636,463.709772,459.100789,482.126589,...,477.541695,521.729294,502.856030,471.002144,485.324483,507.062311,521.457250,490.907687,480.312386,457.177166
1,1019000,499.585391,508.509752,421.748681,446.790894,480.256419,471.587569,457.372453,441.357433,468.003882,...,460.994583,448.315687,476.387563,448.358631,448.451536,479.129280,496.595954,464.294189,445.773821,441.038802
2,1022500,535.796254,560.605325,480.729019,505.535349,519.688975,525.732224,494.877831,497.125226,520.270494,...,504.111010,518.397766,536.849804,502.146776,492.854907,517.617510,532.765595,519.859560,500.135492,476.573745
3,1030500,542.833077,558.592781,484.850538,519.901025,529.423530,534.289118,499.099935,507.452618,528.592844,...,508.964262,534.171812,545.085634,500.798790,510.233407,542.172768,548.714729,517.182321,497.271122,492.144484
4,1031500,527.025566,552.803853,486.754495,514.397505,520.609899,523.494781,512.750708,515.699722,537.828460,...,509.612876,532.145247,548.136173,497.905593,502.271921,536.402295,540.497048,512.987179,509.845377,481.151956


In [21]:
# Merge the DataFrames on Gage_ID (OBJECTID or other unique field)
aligned_data = pd.merge(
    sanford_df,
    conus404_df,
    on="name",  # Change this if a different ID field is needed
    how="inner"
)

# Rename columns for clarity
aligned_data.rename(columns=lambda x: x.replace("AET_", "Sanford_").replace("ET_", "CONUS404_"), inplace=True)

# Drop rows with NaN values
aligned_data = aligned_data.dropna()

# Display first few rows
display(aligned_data.head())


,name,Sanford_1990,Sanford_1991,Sanford_1992,Sanford_1993,Sanford_1994,Sanford_1995,Sanford_1996,Sanford_1997,Sanford_1998,...,CONUS404_2000,CONUS404_2001,CONUS404_2002,CONUS404_2003,CONUS404_2004,CONUS404_2005,CONUS404_2006,CONUS404_2007,CONUS404_2008,CONUS404_2009
0,1013500,532.240341,507.776588,535.722459,539.077791,565.502197,482.285055,566.518196,523.279233,563.964709,...,477.541695,521.729294,502.856030,471.002144,485.324483,507.062311,521.457250,490.907687,480.312386,457.177166
1,1019000,567.941718,582.810206,511.858339,486.392273,502.745313,517.294172,550.425525,542.522675,534.586589,...,460.994583,448.315687,476.387563,448.358631,448.451536,479.129280,496.595954,464.294189,445.773821,441.038802
2,1022500,569.082703,594.823793,492.098908,469.906591,509.816766,511.827485,546.341557,520.660206,522.562560,...,504.111010,518.397766,536.849804,502.146776,492.854907,517.617510,532.765595,519.859560,500.135492,476.573745
3,1030500,567.746502,532.820693,500.170837,510.349288,524.644646,484.188340,566.161814,533.798302,547.532335,...,508.964262,534.171812,545.085634,500.798790,510.233407,542.172768,548.714729,517.182321,497.271122,492.144484
4,1031500,579.718110,555.205357,499.591072,510.097644,548.800909,465.343476,571.734233,534.114904,578.778529,...,509.612876,532.145247,548.136173,497.905593,502.271921,536.402295,540.497048,512.987179,509.845377,481.151956


In [24]:


# Define functions for Bias (BA) and RMSE
def calculate_bias(true_values, predicted_values):
    """Calculate Bias (Systematic Error)."""
    return np.mean(predicted_values - true_values)

def calculate_rmse(true_values, predicted_values):
    """Calculate Root Mean Square Error (RMSE)."""
    return np.sqrt(np.mean((predicted_values - true_values) ** 2))

# Compute Bias and RMSE for each Gage_ID
results = []
for gage_id, group in aligned_data.groupby("name"):
    true_values = group.filter(like="Sanford_").values.flatten()
    predicted_values = group.filter(like="CONUS404_").values.flatten()

    ba = calculate_bias(true_values, predicted_values)
    rmse = calculate_rmse(true_values, predicted_values)

    results.append({"Gage_ID": gage_id, "Bias (BA)": ba, "RMSE": rmse})

# Convert results to DataFrame and save
results_df = pd.DataFrame(results)
results_df.to_csv("AET_BE_RMSE_Sanford-CONUS404.csv", index=False)

# Display results
display(results_df.head())


,Gage_ID,Bias (BA),RMSE
0,1013500,-52.805331,65.574252
1,1019000,-80.406154,88.265015
2,1022500,-23.470314,45.715613
3,1030500,-16.553470,40.406543
4,1031500,-24.925571,45.066570


In [25]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import os

# Create output folder for scatter plots
output_folder = "Sanford_Scatter_Plots"
os.makedirs(output_folder, exist_ok=True)

# Function to create scatter plots with regression
def plot_scatter_with_regression(data, gage_id, output_folder):
    """Generate scatter plot for a specific Gage_ID."""
    gage_data = data[data["name"] == gage_id]

    # Extract X (Sanford ET) and Y (CONUS404 ET)
    X = gage_data.filter(like="Sanford_").values.flatten().reshape(-1, 1)
    Y = gage_data.filter(like="CONUS404_").values.flatten()

    if len(X) > 1:  # Ensure enough data points
        model = LinearRegression()
        model.fit(X, Y)
        Y_pred = model.predict(X)
        r_squared = model.score(X, Y)
        slope = model.coef_[0]

        # Plot Scatter with Regression
        plt.figure(figsize=(8, 6))
        plt.scatter(X, Y, color="blue", label="Data points")
        plt.plot(X, Y_pred, color="red", label=f"Linear Fit (y={slope:.2f}x + {model.intercept_:.2f})")
        plt.xlabel("Sanford AET (mm)")
        plt.ylabel("CONUS404 AET (mm)")
        plt.title(f"Gage {gage_id}: Scatter Plot")
        plt.legend()
        plt.grid(True)
        plt.text(0.05, 0.95, f"R² = {r_squared:.2f}", transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')

        # Save plot
        filename = os.path.join(output_folder, f"Gage_{gage_id}_scatter.png")
        plt.savefig(filename, format="png")
        plt.close()
        print(f"Scatter plot saved for Gage ID: {gage_id} at {filename}")

# Loop through each Gage_ID and generate scatter plots
for gage_id in aligned_data["name"].unique():
    plot_scatter_with_regression(aligned_data, gage_id, output_folder)

print("✅ Scatter plots generated and saved in:", output_folder)


Scatter plot saved for Gage ID: 1013500 at Sanford_Scatter_Plots\Gage_1013500_scatter.png
Scatter plot saved for Gage ID: 1019000 at Sanford_Scatter_Plots\Gage_1019000_scatter.png
Scatter plot saved for Gage ID: 1022500 at Sanford_Scatter_Plots\Gage_1022500_scatter.png
Scatter plot saved for Gage ID: 1030500 at Sanford_Scatter_Plots\Gage_1030500_scatter.png
Scatter plot saved for Gage ID: 1031500 at Sanford_Scatter_Plots\Gage_1031500_scatter.png
Scatter plot saved for Gage ID: 1038000 at Sanford_Scatter_Plots\Gage_1038000_scatter.png
Scatter plot saved for Gage ID: 1047000 at Sanford_Scatter_Plots\Gage_1047000_scatter.png
Scatter plot saved for Gage ID: 1052500 at Sanford_Scatter_Plots\Gage_1052500_scatter.png
Scatter plot saved for Gage ID: 1054200 at Sanford_Scatter_Plots\Gage_1054200_scatter.png
Scatter plot saved for Gage ID: 1055000 at Sanford_Scatter_Plots\Gage_1055000_scatter.png
Scatter plot saved for Gage ID: 1057000 at Sanford_Scatter_Plots\Gage_1057000_scatter.png
Scatter pl

In [27]:
# Convert Regression Results to DataFrame and save
scatter_results = []
for gage_id, group in aligned_data.groupby("name"):
    X = group.filter(like="Sanford_").values.flatten().reshape(-1, 1)
    Y = group.filter(like="CONUS404_").values.flatten()

    if len(X) > 1:  # Ensure enough data points
        model = LinearRegression()
        model.fit(X, Y)
        r_squared = model.score(X, Y)
        slope = model.coef_[0]

        # Store Regression Results
        scatter_results.append({
            "Gage_ID": gage_id,
            "R_Squared": r_squared,
            "Slope": slope,
            "Intercept": model.intercept_
        })

# Save Regression Results to CSV
scatter_results_df = pd.DataFrame(scatter_results)
scatter_results_df.to_csv("Scatter_Regression_Results.csv", index=False)

# Display results
display(scatter_results_df.head())

print("✅ Regression results saved as AET_Scatter_Regression_Results.csv")


,Gage_ID,R_Squared,Slope,Intercept
0,1013500,0.052510,-0.190895,595.843329
1,1019000,0.185523,0.245014,331.400412
2,1022500,0.089416,0.158771,429.035394
3,1030500,0.010477,0.072099,484.373715
4,1031500,0.053024,0.133807,448.291064


✅ Regression results saved as AET_Scatter_Regression_Results.csv


In [28]:
import pandas as pd

# File paths
sanford_file = r"C:\Users\adi10136\OneDrive - Iowa State University\Desktop\CONUS404\ET Sanford\ET_Mean_Sanford_Agreed_Basins\AET_Basins_Mean_Values.csv"
conus404_file = r"C:\Users\adi10136\OneDrive - Iowa State University\Desktop\CONUS404\ET_WaterYears\AET_mean_Basins_CONUS404\ET_Basins_Mean_Values.csv"

# Load the CSV files
sanford_df = pd.read_csv(sanford_file)
conus404_df = pd.read_csv(conus404_file)

# Merge the DataFrames on Gage_ID (OBJECTID or other unique field)
aligned_data = pd.merge(
    sanford_df,
    conus404_df,
    on="name",  # Change this if a different ID field is needed
    how="inner"
)

# Rename columns for clarity
aligned_data.rename(columns=lambda x: x.replace("AET_", "Sanford_").replace("ET_", "CONUS404_"), inplace=True)

# Drop rows with NaN values
aligned_data = aligned_data.dropna()

# Display first few rows
display(aligned_data.head())

,name,Sanford_1990,Sanford_1991,Sanford_1992,Sanford_1993,Sanford_1994,Sanford_1995,Sanford_1996,Sanford_1997,Sanford_1998,...,CONUS404_2000,CONUS404_2001,CONUS404_2002,CONUS404_2003,CONUS404_2004,CONUS404_2005,CONUS404_2006,CONUS404_2007,CONUS404_2008,CONUS404_2009
0,1013500,532.240341,507.776588,535.722459,539.077791,565.502197,482.285055,566.518196,523.279233,563.964709,...,477.541695,521.729294,502.856030,471.002144,485.324483,507.062311,521.457250,490.907687,480.312386,457.177166
1,1019000,567.941718,582.810206,511.858339,486.392273,502.745313,517.294172,550.425525,542.522675,534.586589,...,460.994583,448.315687,476.387563,448.358631,448.451536,479.129280,496.595954,464.294189,445.773821,441.038802
2,1022500,569.082703,594.823793,492.098908,469.906591,509.816766,511.827485,546.341557,520.660206,522.562560,...,504.111010,518.397766,536.849804,502.146776,492.854907,517.617510,532.765595,519.859560,500.135492,476.573745
3,1030500,567.746502,532.820693,500.170837,510.349288,524.644646,484.188340,566.161814,533.798302,547.532335,...,508.964262,534.171812,545.085634,500.798790,510.233407,542.172768,548.714729,517.182321,497.271122,492.144484
4,1031500,579.718110,555.205357,499.591072,510.097644,548.800909,465.343476,571.734233,534.114904,578.778529,...,509.612876,532.145247,548.136173,497.905593,502.271921,536.402295,540.497048,512.987179,509.845377,481.151956


In [31]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

# Function to create scatter plots with linear regression for a single gage
def plot_scatter_with_regression(data, gage_id):
    """Generates regression results for a specific gage ID."""
    gage_data = data[data["name"] == gage_id]

    # Extract X (Sanford AET) and Y (CONUS404 AET)
    X = gage_data.filter(like="Sanford_").values.flatten().reshape(-1, 1)
    Y = gage_data.filter(like="CONUS404_").values.flatten()

    if len(X) > 1:  # Ensure enough data points
        model = LinearRegression()
        model.fit(X, Y)
        Y_pred = model.predict(X)
        r_squared = model.score(X, Y)
        return gage_id, X, Y, Y_pred, r_squared, model, len(X)
    
    return None

# Function to create a grid layout of scatter plots with filtering
def plot_scatter_grid(data, scatter_width=4, scatter_height=3, output_file="scatter_grid_poster.png"):
    """Creates a grid of scatter plots for gages with R² > 0.7 and > 15 data points."""
    filtered_gages = []
    results = []

    for gage_id in data["name"].unique():
        result = plot_scatter_with_regression(data, gage_id)
        if result:
            gage_id, X, Y, Y_pred, r_squared, model, num_points = result
            if r_squared > 0.70 and num_points > 15:  # Apply both filters
                filtered_gages.append(gage_id)
                results.append((X, Y, Y_pred, r_squared, gage_id, model))

    # Determine the number of rows and columns for the grid
    n = len(filtered_gages)
    cols = 4  # Adjust as needed for layout
    rows = (n + cols - 1) // cols  # Calculate number of rows

    # Set figure size based on scatter plot dimensions
    fig, axes = plt.subplots(rows, cols, figsize=(scatter_width * cols, scatter_height * rows))
    axes = axes.flatten() if n > 1 else [axes]

    for i, (X, Y, Y_pred, r_squared, gage_id, model) in enumerate(results):
        ax = axes[i]
        ax.scatter(X, Y, color="black", alpha=0.7, s=9, label="Data points")
        ax.plot(X, Y_pred, color="blue", label=f"Fit: y={model.coef_[0]:.2f}x+{model.intercept_:.2f}")
        ax.plot([min(X), max(X)], [min(X), max(X)], linestyle="--", color="gray", label="1:1 Line")
        ax.set_title(f"Gage: {gage_id}\nR² = {r_squared:.2f}", fontsize=18)
        ax.set_xlabel("Sanford AET (mm)", fontsize=16)
        ax.set_ylabel("CONUS404 AET (mm)", fontsize=16)
        ax.tick_params(axis="y", which="major", labelsize=14)
        ax.tick_params(axis="x", which="major", labelsize=14)
        ax.legend(fontsize=12, loc="lower right")

    # Hide unused subplots if any
    for j in range(len(results), len(axes)):
        axes[j].axis("off")

    # Adjust layout for presentation
    plt.tight_layout(pad=2.0)
    plt.savefig(output_file, dpi=600)
    plt.close()

    print(f"Scatter grid saved to {output_file}")

# Generate the scatter plot grid for gages with R² > 0.7
plot_scatter_grid(aligned_data, scatter_width=5, scatter_height=4, output_file="AET_CONUS404_gages_SP.png")


Scatter grid saved to AET_CONUS404_gages_SP.png
